# Data Combiner

Here we try to bring together all the data into one clean dataframe

In [1]:
# !pip install pandas_profiling
# !pip install ipywidgets

In [2]:
import pandas as pd
import os
import plotly.express as px
import numpy as np

In [3]:
from pandas_profiling import ProfileReport
from statistics import mean
from scipy.stats import f_oneway


In [4]:
result = pd.read_csv(r"G:\Meine Ablage\HSLU\Master\Master Thesis\Coding Area\Data\result.csv", index_col=0)
result_normal = result[result["Report Type"]=="Normal Report"]

In [5]:
result_normal[0:1]

,Company,Label,Fulltext Cosine Similarity Wiki Summary,Fulltext Cosine Similarity Human Rights,Zero Shot Fulltext,PDF Name,TF-IDF,Top N Words,Report Type,Sentence Cosine Similarity Wiki Summarized,Sentence Cosine Similarity Wiki Summarized Mean,Sentence Cosine Similarity Wiki Summarized Mean Threshold 0,Sentence by Sentence Cosine Similarity human rights,Sentence by Sentence Cosine Similarity human rights mean,Sentence by Sentence Cosine Similarity human rights Mean Threshold 0,Zero Shot Learning Sentence Raw,Classified Text,Zero Shot Learning Sentence Mean,Zero Shot Learning Sentence Mean Threshold
0,child_cocoa_school,sustainability,0.54579,0.54579,0.242261,nestle-tackling-child-labor-report-2019-en,"[('child', 0.3676919823097556), ('report', 0.2...","[('child', 391), ('cocoa', 146), ('school', 14...",Normal Report,"[tensor([[0.0519]]), tensor([[0.0648]]), tenso...",0.065,0.11,"[tensor([[0.0519]]), tensor([[0.0648]]), tenso...",0.065,0.11,"[0.7845346331596375, 0.8715912103652954, 0.683...",['Tackling Child Labor Report Continuing our c...,0.501,0.501


### 5.1.3	Sentence-BERT Similarität satzweise vs. Zero-Shot Learning Fulltext

In [6]:
fig = px.scatter(result, x="Sentence Cosine Similarity Wiki Summarized Mean", y="Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", trendline="ols", color = "Report Type")
fig.show()
results = px.get_trendline_results(fig)
print(results)
results.px_fit_results.iloc[0].summary()


     Report Type                                     px_fit_results
0  Normal Report  <statsmodels.regression.linear_model.Regressio...
1             DE  <statsmodels.regression.linear_model.Regressio...
2           Fake  <statsmodels.regression.linear_model.Regressio...


In [21]:
result_2 = result[result["Label"] == "fraud"]

In [34]:

fig = px.scatter(result_normal, x="Zero Shot Learning Sentence Mean Threshold", y="Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", trendline="ols")
fig.show()
results = px.get_trendline_results(fig)
print(results)
results.px_fit_results.iloc[0].summary()


                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.576
Method:                 Least Squares   F-statistic:                     734.1
Date:                Sun, 11 Dec 2022   Prob (F-statistic):          1.31e-102
Time:                        18:01:49   Log-Likelihood:                 1143.1
No. Observations:                 540   AIC:                            -2282.
Df Residuals:                     538   BIC:                            -2274.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0619      0.002     29.077      0.000       0.058       0.066
x1             0.1935      0.007     27.094      0.000       0.179       0.208
==============================================================================
Omnibus:                      159.564   Durbin-Watson:                   1.464
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1079.553
Skew:                           1.114   Prob(JB):                    3.78e-235
Kurtosis:                       9.559   Cond. No.                         6.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:

result_prep = result[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", "Zero Shot Learning Sentence Mean Threshold"]]
result_prep_melt = result_prep.melt(var_name="Data")
fig = px.box(result_prep_melt, y="value", points="all", color = "Data", width=900)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=1,
    xanchor="left",
    x=0.01
))
fig.show()


In [9]:
result_prep = result[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", "Zero Shot Fulltext", "Label"]]

croped_results = result_prep[result_prep["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0"] <= 0.179]
croped_results = croped_results[croped_results["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0"] >= 0.032]
croped_results = croped_results[croped_results["Zero Shot Fulltext"] <= 0.373]


In [10]:

result_prep = croped_results[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", "Zero Shot Fulltext"]]
result_prep_melt = result_prep.melt(var_name="Data")
fig = px.box(result_prep_melt, y="value", points="all", color = "Data")
fig.show()



            Label                                     px_fit_results
0  sustainability  <statsmodels.regression.linear_model.Regressio...
1    human rights  <statsmodels.regression.linear_model.Regressio...
2           fraud  <statsmodels.regression.linear_model.Regressio...
3   social issues  <statsmodels.regression.linear_model.Regressio...
4      labour law  <statsmodels.regression.linear_model.Regressio...


In [ ]:
fig = px.scatter(croped_results, x="Sentence Cosine Similarity Wiki Summarized Mean Threshold 0", y="Zero Shot Fulltext", trendline="ols", color = "Label")
fig.show()
results = px.get_trendline_results(fig)
print(results)

In [11]:

results.px_fit_results.iloc[0].summary()
result_prep_melt_threshold = result[["Sentence Cosine Similarity Wiki Summarized Mean Threshold 0"] ]
result_prep_melt_no_threshold = result[["Sentence Cosine Similarity Wiki Summarized Mean"] ]
result_prep_melt_no_threshold[0:1]
f_oneway(result_prep_melt_threshold, result_prep_melt_no_threshold)


F_onewayResult(statistic=array([172.31546366]), pvalue=array([8.10591615e-37]))